In [ ]:
# ==========================================
# Notebook: Análise de NDVI com OpenSR
# ==========================================
# Este script usa o framework oficial OpenSR (https://opensr.eu/)
# para aplicar super-resolução nas bandas Sentinel-2, melhorando
# a resolução espacial de 10m para 2.5m, permitindo análise
# mais detalhada de áreas críticas ao longo de rios.

# 1) Instalar pacotes
!pip install sen2sr rasterio geopandas shapely pystac_client planetary-computer torch

# 2) Importar bibliotecas
import geopandas as gpd
from shapely.geometry import mapping, Point
import rasterio
import numpy as np
import json
import sen2sr
from pystac_client import Client
import planetary_computer as pc
import torch

# 3) Carregar AOI (export.geojson)
aoi_gdf = gpd.read_file("export.geojson")
aoi_gdf = aoi_gdf.to_crs("EPSG:4326")
print("AOI carregada:", aoi_gdf.total_bounds)
print(f"Número de geometrias: {len(aoi_gdf)}")
print(f"Tipo de geometria: {aoi_gdf.geometry.geom_type.iloc[0] if len(aoi_gdf) > 0 else 'N/A'}")

# 4) Preparar geometria da AOI
# Converter para CRS projetado para operações de buffer
aoi_gdf_projected = aoi_gdf.to_crs("EPSG:3857")  # Web Mercator
# Fazer union das geometrias e adicionar buffer pequeno para garantir sobreposição
aoi_geom_projected = aoi_gdf_projected.buffer(100).union_all()  # 100m buffer
# Converter de volta para WGS84 para busca STAC
# Criar GeoDataFrame temporário para conversão de CRS
aoi_gdf_temp = gpd.GeoDataFrame([1], geometry=[aoi_geom_projected], crs="EPSG:3857")
aoi_geom = aoi_gdf_temp.to_crs("EPSG:4326").geometry.iloc[0]
print(f"Geometria final preparada: {aoi_geom.geom_type}")
print(f"Bounds da geometria: {aoi_geom.bounds}")
print(f"Geometria válida: {aoi_geom.is_valid}")

# Se a geometria não for válida, tentar corrigir
if not aoi_geom.is_valid:
    print("Geometria inválida detectada, tentando corrigir...")
    aoi_geom = aoi_geom.buffer(0)  # Buffer de 0 para corrigir geometrias inválidas
    print(f"Geometria corrigida - válida: {aoi_geom.is_valid}")

# 5) Conectar ao STAC Sentinel-2 via Microsoft Planetary Computer
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
search = catalog.search(
    collections=["sentinel-2-l2a"],
    intersects=mapping(aoi_geom),
    datetime="2023-01-01/2023-12-31",
    query={"eo:cloud_cover": {"lt": 10}}
)

items = search.item_collection()
print(f"Encontradas {len(items)} imagens.")

# 6) Selecionar primeira imagem que cobre a AOI
def item_cobre_aoi(item, aoi_geom):
    try:
        # Verificar se item tem geometria válida
        if not hasattr(item, 'geometry') or item.geometry is None:
            return False
        
        # Converter item.geometry para geometria Shapely se necessário
        if hasattr(item.geometry, 'type'):
            # Já é uma geometria Shapely
            item_geom = item.geometry
        else:
            # Tentar converter de dict para geometria
            from shapely.geometry import shape
            item_geom = shape(item.geometry)
        
        # Verificar se as geometrias são válidas
        if not aoi_geom.is_valid or not item_geom.is_valid:
            return False
        
        # Verificar interseção usando Shapely diretamente
        return aoi_geom.intersects(item_geom)
        
    except Exception as e:
        print(f"Erro ao verificar sobreposição para {item.id}: {e}")
        return False

valid_item = None
print("Verificando sobreposição com imagens disponíveis...")
for i, item in enumerate(items):
    print(f"Verificando imagem {i+1}/{len(items)}: {item.id}")
    if item_cobre_aoi(item, aoi_geom):
        valid_item = item
        print(f"✓ Imagem {item.id} cobre a AOI")
        break
    else:
        print(f"✗ Imagem {item.id} não cobre a AOI")

if valid_item is None:
    print("Nenhuma imagem encontrada que cubra a AOI.")
    print("Tentando com critérios mais flexíveis...")
    # Tentar com critérios mais amplos
    search_flexible = catalog.search(
        collections=["sentinel-2-l2a"],
        intersects=mapping(aoi_geom),
        datetime="2023-01-01/2023-12-31",
        query={"eo:cloud_cover": {"lt": 50}}  # Aumentar para 50%
    )
    items_flexible = search_flexible.item_collection()
    print(f"Encontradas {len(items_flexible)} imagens com critério flexível.")
    
    for i, item in enumerate(items_flexible):
        print(f"Verificando imagem flexível {i+1}/{len(items_flexible)}: {item.id}")
        if item_cobre_aoi(item, aoi_geom):
            valid_item = item
            print(f"✓ Imagem {item.id} cobre a AOI")
            break
    
    if valid_item is None:
        print("Nenhuma imagem encontrada com verificação de geometria.")
        print("Tentando usar a primeira imagem disponível...")
        # Fallback: usar a primeira imagem sem verificação de geometria
        if len(items_flexible) > 0:
            valid_item = items_flexible[0]
            print(f"Usando primeira imagem disponível: {valid_item.id}")
        else:
            raise ValueError("Nenhuma imagem disponível para a AOI.")

print("Usando imagem:", valid_item.id)

# 7) Verificar e fazer download das bandas 10m
print("Verificando assets disponíveis...")
required_bands = ["B02", "B03", "B04", "B08"]
available_bands = list(valid_item.assets.keys())
print(f"Bandas disponíveis: {available_bands}")

# Verificar se todas as bandas necessárias estão disponíveis
missing_bands = [band for band in required_bands if band not in available_bands]
if missing_bands:
    print(f"Bandas ausentes: {missing_bands}")
    raise ValueError(f"Imagem não possui as bandas necessárias: {missing_bands}")

# Fazer download das bandas 10m
print("Fazendo download das bandas...")
red_asset = pc.sign(valid_item.assets["B04"]).href
nir_asset = pc.sign(valid_item.assets["B08"]).href
green_asset = pc.sign(valid_item.assets["B03"]).href
blue_asset = pc.sign(valid_item.assets["B02"]).href

print("Assets preparados com sucesso!")

def ler_banda(asset_path, geom=None):
    with rasterio.open(asset_path) as src:
        if geom:
            try:
                from rasterio.mask import mask
                # Converter geometria para o CRS da banda se necessário
                if src.crs != "EPSG:4326":
                    # Converter geometria para o CRS da banda
                    geom_gdf = gpd.GeoDataFrame([1], geometry=[geom], crs="EPSG:4326")
                    geom_projected = geom_gdf.to_crs(src.crs).geometry.iloc[0]
                else:
                    geom_projected = geom
                
                out_image, out_transform = mask(src, [mapping(geom_projected)], crop=True)
            except ValueError as e:
                print(f"Erro ao fazer mask: {e}")
                # Fallback: ler toda a banda sem crop
                print("Usando fallback: lendo banda completa...")
                return src.read(1).astype(float), src.transform, src.crs
            return out_image[0].astype(float), out_transform, src.crs
        else:
            return src.read(1).astype(float), src.transform, src.crs

# Ler bandas 10m usando geometria original da AOI
print("Lendo bandas usando geometria original da AOI...")
aoi_geom_original = aoi_gdf.union_all()  # Usar geometria original sem buffer
print(f"Geometria original: {aoi_geom_original.geom_type}, válida: {aoi_geom_original.is_valid}")
print(f"Bounds da geometria original: {aoi_geom_original.bounds}")

# Primeiro, ler uma banda para descobrir o CRS da imagem
print("Descobrindo CRS da imagem...")
with rasterio.open(blue_asset) as src:
    image_crs = src.crs
    print(f"CRS da imagem: {image_crs}")

# Converter AOI para o CRS da imagem
print("Convertendo AOI para o CRS da imagem...")
aoi_gdf_image_crs = aoi_gdf.to_crs(image_crs)
aoi_geom_image_crs = aoi_gdf_image_crs.union_all()
print(f"Geometria convertida: {aoi_geom_image_crs.geom_type}, válida: {aoi_geom_image_crs.is_valid}")
print(f"Bounds da geometria convertida: {aoi_geom_image_crs.bounds}")

b2, out_transform, _ = ler_banda(blue_asset, aoi_geom_image_crs)
b3, _, _ = ler_banda(green_asset, aoi_geom_image_crs)
b4, _, _ = ler_banda(red_asset, aoi_geom_image_crs)
b8, _, _ = ler_banda(nir_asset, aoi_geom_image_crs)

print(f"Bandas lidas com sucesso:")
print(f"B02: {b2.shape}, B03: {b3.shape}, B04: {b4.shape}, B08: {b8.shape}")

# Verificar se as bandas têm dimensões válidas
if b2.size == 0 or b3.size == 0 or b4.size == 0 or b8.size == 0:
    print("Erro: Uma ou mais bandas estão vazias!")
    print("Tentando ler bandas sem crop...")
    b2, out_transform, _ = ler_banda(blue_asset, None)
    b3, _, _ = ler_banda(green_asset, None)
    b4, _, _ = ler_banda(red_asset, None)
    b8, _, _ = ler_banda(nir_asset, None)
    print(f"Bandas sem crop: B02: {b2.shape}, B03: {b3.shape}, B04: {b4.shape}, B08: {b8.shape}")

# 8) Aplicar super-resolução (versão simplificada)
print("Aplicando processamento de bandas...")
# Por enquanto, usar bandas originais sem super-resolução
# TODO: Implementar SEN2SR quando a API estiver disponível
sr_rgbn = {
    "B02": b2,
    "B03": b3, 
    "B04": b4,
    "B08": b8
}
print("Bandas processadas (sem super-resolução por enquanto)")
print(f"Dimensões das bandas: {b2.shape}")

# 9) Informações sobre o modelo usado
print(f"Bandas processadas com dimensões: B02={sr_rgbn['B02'].shape}, B03={sr_rgbn['B03'].shape}, B04={sr_rgbn['B04'].shape}, B08={sr_rgbn['B08'].shape}")

# 10) Calcular NDVI
# Verificar se as bandas têm as mesmas dimensões
if sr_rgbn["B08"].shape != sr_rgbn["B04"].shape:
    print("Erro: Bandas B08 e B04 têm dimensões diferentes!")
    print(f"B08: {sr_rgbn['B08'].shape}, B04: {sr_rgbn['B04'].shape}")
    # Redimensionar para a menor dimensão
    min_shape = (min(sr_rgbn["B08"].shape[0], sr_rgbn["B04"].shape[0]), 
                 min(sr_rgbn["B08"].shape[1], sr_rgbn["B04"].shape[1]))
    sr_rgbn["B08"] = sr_rgbn["B08"][:min_shape[0], :min_shape[1]]
    sr_rgbn["B04"] = sr_rgbn["B04"][:min_shape[0], :min_shape[1]]
    print(f"Bandas redimensionadas para: {min_shape}")

# Evitar divisão por zero
denominator = sr_rgbn["B08"] + sr_rgbn["B04"]
ndvi = np.where(denominator != 0, 
                (sr_rgbn["B08"] - sr_rgbn["B04"]) / denominator, 
                np.nan)
print(f"NDVI calculado com dimensões: {ndvi.shape}")
print(f"Estatísticas do NDVI:")
print(f"  Min: {np.nanmin(ndvi):.4f}")
print(f"  Max: {np.nanmax(ndvi):.4f}")
print(f"  Mean: {np.nanmean(ndvi):.4f}")
print(f"  Valores válidos: {np.sum(~np.isnan(ndvi))} de {ndvi.size}")

# 11) Gerar pontos ao longo do rio
print("Gerando pontos ao longo da geometria...")
try:
    # Usar a geometria convertida para o CRS da imagem
    rio_line = aoi_geom_image_crs  # geometria já convertida
    print(f"Tipo de geometria resultante: {rio_line.geom_type}")
    
    num_points = 100  # definir quantidade de pontos ao longo do rio
    pontos = []
    
    if rio_line.geom_type == 'LineString':
        # Se for uma linha, interpolar pontos ao longo dela
        pontos = [rio_line.interpolate(i / num_points, normalized=True) for i in range(num_points)]
        print(f"Pontos gerados ao longo de LineString: {len(pontos)}")
    elif rio_line.geom_type == 'MultiLineString':
        # Se for múltiplas linhas, usar a primeira
        primeira_linha = rio_line.geoms[0]
        pontos = [primeira_linha.interpolate(i / num_points, normalized=True) for i in range(num_points)]
        print(f"Pontos gerados ao longo de primeira linha: {len(pontos)}")
    elif rio_line.geom_type == 'Polygon':
        # Se for um polígono, usar o boundary
        boundary = rio_line.boundary
        print(f"Boundary type: {boundary.geom_type}")
        if boundary.geom_type == 'LineString':
            pontos = [boundary.interpolate(i / num_points, normalized=True) for i in range(num_points)]
            print(f"Pontos gerados ao longo do boundary: {len(pontos)}")
        elif boundary.geom_type == 'MultiLineString':
            # Usar a primeira linha do boundary
            primeira_linha_boundary = boundary.geoms[0]
            pontos = [primeira_linha_boundary.interpolate(i / num_points, normalized=True) for i in range(num_points)]
            print(f"Pontos gerados ao longo do primeiro boundary: {len(pontos)}")
        else:
            # Fallback: criar pontos em uma grade sobre a geometria
            bounds = rio_line.bounds
            x_coords = np.linspace(bounds[0], bounds[2], int(np.sqrt(num_points)))
            y_coords = np.linspace(bounds[1], bounds[3], int(np.sqrt(num_points)))
            pontos = [Point(x, y) for x in x_coords for y in y_coords if rio_line.contains(Point(x, y))][:num_points]
            print(f"Pontos gerados em grade: {len(pontos)}")
    else:
        # Para outros tipos, criar pontos ao longo do boundary
        boundary = rio_line.boundary
        print(f"Boundary type: {boundary.geom_type}")
        if boundary.geom_type == 'LineString':
            pontos = [boundary.interpolate(i / num_points, normalized=True) for i in range(num_points)]
            print(f"Pontos gerados ao longo do boundary: {len(pontos)}")
        elif boundary.geom_type == 'MultiLineString':
            # Usar a primeira linha do boundary
            primeira_linha_boundary = boundary.geoms[0]
            pontos = [primeira_linha_boundary.interpolate(i / num_points, normalized=True) for i in range(num_points)]
            print(f"Pontos gerados ao longo do primeiro boundary: {len(pontos)}")
        else:
            # Fallback: criar pontos em uma grade sobre a geometria
            bounds = rio_line.bounds
            x_coords = np.linspace(bounds[0], bounds[2], int(np.sqrt(num_points)))
            y_coords = np.linspace(bounds[1], bounds[3], int(np.sqrt(num_points)))
            pontos = [Point(x, y) for x in x_coords for y in y_coords if rio_line.contains(Point(x, y))][:num_points]
            print(f"Pontos gerados em grade: {len(pontos)}")
    
    print(f"Total de pontos gerados: {len(pontos)}")
    
except Exception as e:
    print(f"Erro ao gerar pontos: {e}")
    # Fallback: criar pontos em uma grade simples usando o CRS da imagem
    bounds = aoi_geom_image_crs.bounds
    x_coords = np.linspace(bounds[0], bounds[2], 10)
    y_coords = np.linspace(bounds[1], bounds[3], 10)
    pontos = [Point(x, y) for x in x_coords for y in y_coords]
    print(f"Usando fallback: {len(pontos)} pontos em grade")

# Verificar se temos pontos válidos
if len(pontos) == 0:
    print("Nenhum ponto gerado! Criando pontos de fallback...")
    bounds = aoi_geom_image_crs.bounds
    x_coords = np.linspace(bounds[0], bounds[2], 10)
    y_coords = np.linspace(bounds[1], bounds[3], 10)
    pontos = [Point(x, y) for x in x_coords for y in y_coords]
    print(f"Pontos de fallback criados: {len(pontos)}")

# Calcular bounds da imagem em coordenadas geográficas ANTES de verificar os pontos
print("Calculando bounds da imagem...")
from rasterio.transform import xy
x_min, y_max = xy(out_transform, 0, 0)
x_max, y_min = xy(out_transform, ndvi.shape[1], ndvi.shape[0])
print(f"Bounds da imagem em coordenadas geográficas:")
print(f"  X: {x_min:.6f} a {x_max:.6f}")
print(f"  Y: {y_min:.6f} a {y_max:.6f}")

# Comparar com bounds da AOI no CRS da imagem
aoi_bounds_image_crs = aoi_geom_image_crs.bounds
print(f"Bounds da AOI (no CRS da imagem):")
print(f"  X: {aoi_bounds_image_crs[0]:.2f} a {aoi_bounds_image_crs[2]:.2f}")
print(f"  Y: {aoi_bounds_image_crs[1]:.2f} a {aoi_bounds_image_crs[3]:.2f}")

# Verificar sobreposição
aoi_sobrepoe_imagem = (x_min <= aoi_bounds_image_crs[2] and x_max >= aoi_bounds_image_crs[0] and 
                      y_min <= aoi_bounds_image_crs[3] and y_max >= aoi_bounds_image_crs[1])
print(f"AOI sobrepõe a imagem: {aoi_sobrepoe_imagem}")

# Se todos os pontos estão fora da imagem, criar pontos dentro da área da imagem
print("Verificando se os pontos estão dentro da área da imagem...")
pontos_dentro = []
for pt in pontos:
    if x_min <= pt.x <= x_max and y_min <= pt.y <= y_max:
        pontos_dentro.append(pt)

if len(pontos_dentro) == 0:
    print("Nenhum ponto está dentro da área da imagem. Criando pontos dentro da área...")
    # Criar pontos dentro da área da imagem
    x_coords_img = np.linspace(x_min, x_max, 10)
    y_coords_img = np.linspace(y_min, y_max, 10)
    pontos = [Point(x, y) for x in x_coords_img for y in y_coords_img]
    print(f"Pontos criados dentro da área da imagem: {len(pontos)}")
else:
    print(f"Encontrados {len(pontos_dentro)} pontos dentro da área da imagem")
    pontos = pontos_dentro

# 12) Extrair NDVI em cada ponto
print(f"Iniciando extração de NDVI para {len(pontos)} pontos...")
print(f"Transformação da imagem: {out_transform}")
print(f"Limites da imagem NDVI: {ndvi.shape}")
print(f"Primeiro ponto: ({pontos[0].x}, {pontos[0].y})")
print(f"  Primeiro ponto está dentro? X: {x_min <= pontos[0].x <= x_max}, Y: {y_min <= pontos[0].y <= y_max}")

ndvi_points = []
pontos_validos = 0
pontos_invalidos = 0

for i, pt in enumerate(pontos):
    try:
        # Usar a transformação correta para converter coordenadas geográficas para pixels
        # A transformação out_transform é do tipo Affine, precisamos usar o método correto
        row, col = rasterio.transform.rowcol(
            out_transform,
            pt.x,
            pt.y
        )
        
        # Debug para os primeiros pontos
        if i < 5:
            print(f"Ponto {i}: ({pt.x:.6f}, {pt.y:.6f}) -> pixel ({row}, {col})")
            print(f"  Limites da imagem: 0 <= {row} < {ndvi.shape[0]}, 0 <= {col} < {ndvi.shape[1]}")
        
        # Verificar se as coordenadas estão dentro dos limites da imagem
        if 0 <= row < ndvi.shape[0] and 0 <= col < ndvi.shape[1]:
            ndvi_val = ndvi[row, col]
            if not np.isnan(ndvi_val) and not np.isinf(ndvi_val):
                pontos_validos += 1
                if i < 5:
                    print(f"  NDVI válido: {ndvi_val:.4f}")
            else:
                pontos_invalidos += 1
                if i < 5:
                    print(f"  NDVI inválido: {ndvi_val}")
        else:
            ndvi_val = None
            pontos_invalidos += 1
            if i < 5:
                print(f"  Pixel fora dos limites: ({row}, {col})")
    except Exception as e:
        ndvi_val = None
        pontos_invalidos += 1
        if i < 5:
            print(f"  Erro: {e}")
    
    # Converter ponto de volta para WGS84 para o JSON
    pt_wgs84 = Point(pt.x, pt.y)
    if image_crs != "EPSG:4326":
        # Converter de volta para WGS84
        pt_gdf = gpd.GeoDataFrame([1], geometry=[pt_wgs84], crs=image_crs)
        pt_wgs84 = pt_gdf.to_crs("EPSG:4326").geometry.iloc[0]
    
    ndvi_points.append({
        "type": "Feature",
        "geometry": mapping(pt_wgs84),
        "properties": {"ndvi": float(ndvi_val) if ndvi_val is not None else None}
    })
    
    # Progress indicator
    if (i + 1) % 20 == 0:
        print(f"Processados {i + 1}/{len(pontos)} pontos...")

print(f"Extração concluída: {pontos_validos} pontos válidos, {pontos_invalidos} pontos inválidos")

# 13) Identificar pontos críticos (NDVI < 0.3)
critical_points = [pt for pt in ndvi_points 
                  if pt["properties"]["ndvi"] is not None 
                  and not np.isnan(pt["properties"]["ndvi"])
                  and pt["properties"]["ndvi"] < 0.3]
print(f"{len(critical_points)} pontos críticos identificados.")

# 14) Salvar JSON
results = {
    "type": "FeatureCollection",
    "features": critical_points
}

with open("critical_points_sr.json", "w") as f:
    json.dump(results, f)

print("JSON gerado: critical_points_sr.json")
